# Setup



## IMPORTANT

Before you can access the data, you need to save the folder "E442-Final-Project-Data" to your personal drive.

To acces the data, right click the folder "E442-Final-Project-Data" in the same directory as this .ipynb and do this: 

> "Add shortcut to Drive" --> "My Drive" --> "ADD SHORTCUT"

It must be in the root directory of your google drive.

## Imports

In [ ]:
# For mounting drive
from google.colab import drive

#For loading and saving data
from os.path import exists
import pickle
!pip install mat73;
import mat73;


# For Splitting data
import random
import math

# Other
import cv2
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import torchvision
from torchvision import datasets, models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchsummary import summary

from skimage.transform import resize
from skimage import transform

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu");

# New Section

## Mount Data

In [ ]:
drive.mount('/content/drive/')
path = 'drive/MyDrive/E442-Final-Project-Data/nyu_depth_data_labeled.mat'

## Load Data

We use the NYUDepth dataset which can be found [here](https://cs.nyu.edu/~silberman/datasets/nyu_depth_v1.html). There is a NYUDepth v2 that we could potentially use.

In [ ]:
data_dict = mat73.loadmat(path)

In [ ]:
# Access different data via dictionary
print("Keys:", list(data_dict.keys()))
print("Depth Image Shape:", data_dict['depths'].shape)
print("Raw Image Shape:", data_dict['images'].shape)
print("Raw Depth Shape:", data_dict['rawDepths'].shape)

## Training/Testing Splits

In [ ]:
# Initialize idxs for all data
random.seed(0)
data_size = data_dict['depths'].shape[-1]
idxs = [i for i in range(data_size)]
random.shuffle(idxs)

# Parameters for splitting
val_percent = 0.2
small_size = 50
med_size = 350
large_size = 800
full_size = data_size

# Split for small dataset
num_train_small = math.ceil(small_size * (1 - val_percent))
num_val_small = math.floor(small_size * val_percent)
train_small = idxs[:num_train_small]
val_small = idxs[num_train_small : num_train_small + num_val_small]

# Split for medium dataset
num_train_med = math.ceil(med_size * (1 - val_percent))
num_val_med = math.floor(med_size * val_percent)
train_med = idxs[:num_train_med]
val_med = idxs[num_train_med : num_train_med + num_val_med]

# Split for large dataset
num_train_large = math.ceil(large_size * (1 - val_percent))
num_val_large = math.floor(large_size * val_percent)
train_large = idxs[:num_train_large]
val_large = idxs[num_train_large : num_train_large + num_val_large]

# Split for full dataset
num_train_full = math.ceil(full_size * (1 - val_percent))
num_val_full = math.floor(full_size * val_percent)
train_full = idxs[:num_train_full]
val_full = idxs[num_train_full : num_train_full + num_val_full]

# Networks

## Coarse Network

See [our docs](https://docs.google.com/document/d/1rOpBCf5OhNuCuYNpwQqpdJqtWRRBqfEm73pKvBXMSSo/edit) for details about the networks implementation

In [ ]:
class Coarse_Network(nn.Module):
  """
  See our docs for details about the networks implementation
  """

  def __init__(self, dataset="NYUDepth"):
    """
    Builds the final layers 6 & 7 for the coarse network

    This network was constructed from Fig. 1 of the paper, and the following
    quote found in the same section:

      "The coarse-scale network contains five feature extraction layers of
        convolution and max-pooling, followed by two fully connected layers."
    """
    super(Coarse_Network, self).__init__()
    self.dataset = dataset

    # Build layers 1-5
    self.coarse1 = nn.Sequential(
      nn.Conv2d(3, 96, kernel_size=11, stride=4,padding=2),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2))
    self.coarse2 = nn.Sequential(
      nn.Conv2d(96, 256, kernel_size=5, padding=2),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2))
    self.coarse3 = nn.Sequential(
      nn.Conv2d(256, 384, kernel_size=3,padding=1),
      nn.ReLU())
    self.coarse4 = nn.Sequential(
      nn.Conv2d(384, 384, kernel_size=3,padding=1),
      nn.ReLU())
    self.coarse5 = nn.Sequential(
      nn.Conv2d(384, 256, kernel_size=3,stride=2), #was padding=(0,1)
      nn.ReLU())

    # Build layers 6-7
    if self.dataset == "NYUDepth":
      self.coarse6 = nn.Sequential(
        nn.Linear(12288, 4096), # 12288 is 8x6x256
        nn.ReLU(),
        nn.Dropout(0.5)
      )
      self.output = nn.Sequential(
        nn.Linear(4096, 4070) # 4070 is 74x55
      )
    elif self.dataset == "something else":
      pass
    else:
      raise ValueError("No valid dataset passed")

    # Init weights & biases
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        m.bias.data.fill_(0)
      elif isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0)

  def forward(self, x): 
    """
    Forward pass through coarse network 
    """
    # Pass image through first 5 layers
    x = self.coarse1(x)
    x = self.coarse2(x)
    x = self.coarse3(x)
    x = self.coarse4(x)
    x = self.coarse5(x)
    # Pass image through last 2 layers
    #x = torch.flatten(x, 1)
    x = x.reshape(x.size(0), -1)
    if self.dataset == "NYUDepth":
      x = self.coarse6(x)
      x = self.output(x)
      x = x.reshape(x.size(0), 55, 74) # 6 is batch size

    else:
      raise ValueError("No valid dataset passed")
   
    return x


## Fine Network

In [ ]:
class Fine_Network(nn.Module):
  def __init__(self, dataset="NYUDepth"):
    """
    Builds the layers for the fine network

    This network was constructed from Fig. 1 of the paper

    TODO:
      - Confused by "The last convolutional layer is linear" in paper
    """
    super(Fine_Network, self).__init__()
    self.dataset = dataset

    # Build layers 1 and 3
    self.fine1 = nn.Sequential(
      nn.Conv2d(3, 63, kernel_size=9, stride=2),
      nn.ReLU(),
      nn.MaxPool2d(kernel_size=2))
    
    self.fine3 = nn.Sequential(
      nn.Conv2d(64, 64, kernel_size=5,padding=2),
      nn.ReLU())

    # Build layer 4
    if self.dataset == "NYUDepth":
      self.fine4 = nn.Sequential(
          nn.Conv2d(64, 1, kernel_size=5,padding=2))
    elif self.dataset == "something else":
      pass
    else:
      raise ValueError("No valid dataset passed")

    # Init weights & biases
    for m in self.modules():
      if isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, nonlinearity='relu')
        m.bias.data.fill_(0)
      elif isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        m.bias.data.fill_(0)

  def forward(self, x, coarse_out):
    """
    Forward pass through the fine network
    """
    # Pass image through first layer
    x = self.fine1(x)

    # Concat output of coarse network to output of fine1
    x = torch.cat((x,coarse_out),dim=1)

    # Pass image through third layer
    x = self.fine3(x)
   
    # Pass image through final layer
    if self.dataset == "NYUDepth":
      x = self.fine4(x)
      #x = x.view(-1, 55, 74) # 6 is batch size
    else:
      raise ValueError("No valid dataset passed")
   
    return x

# Train Networks

In [ ]:
def display_fine_epoch(input, coarse_output, fine_output, GT):
  rows = 1
  columns = 4 
  fig = plt.figure(figsize=(12, 5))

  # Adds a subplot at the 1st position
  fig.add_subplot(rows, columns, 1)
  
  # Show image
  plt.imshow(cv2.cvtColor(input, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title("Input")
  
  # Adds a subplot at the 2nd position
  fig.add_subplot(rows, columns, 2)
  
  # Show image
  plt.imshow(coarse_output,cmap='jet_r')
  plt.axis('off')
  plt.title("Coarse Output")

  # Adds a subplot at the 3rd position
  fig.add_subplot(rows, columns, 3)

  # Show image
  plt.imshow(fine_output,cmap='jet_r')
  plt.axis('off')
  plt.title("Fine Output")

  # Adds a subplot at the 4th position
  fig.add_subplot(rows, columns, 4)
  
  # Show image
  plt.imshow(GT,cmap='jet_r')
  plt.axis('off')
  plt.title("Ground Truth")

  plt.show()

def display_coarse_epoch(input, coarse_output, GT):
  rows = 1
  columns = 3 
  fig = plt.figure(figsize=(10, 7))

  # Adds a subplot at the 1st position
  fig.add_subplot(rows, columns, 1)
  
  # Show image
  plt.imshow(cv2.cvtColor(input, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  plt.title("Input")
  
  # Adds a subplot at the 2nd position
  fig.add_subplot(rows, columns, 2)
  
  # Show image
  plt.imshow(coarse_output,cmap='jet_r')
  plt.axis('off')
  plt.title("Coarse Output")

  # Adds a subplot at the 3rd position
  fig.add_subplot(rows, columns, 3)

  # Show image
  plt.imshow(GT,cmap='jet_r')
  plt.axis('off')
  plt.title("Ground Truth")

  plt.show()

## Coarse Train

In [ ]:
def train_coarse(coarse_net, dataloaders, criterion, optimizer, num_epochs):

  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.1,patience = 10)
  epoch_loss_history = []
  coarse_net.train()
  for epoch in range(num_epochs):
    running_loss = 0.0
    
    # Images to be printed per epoch
    input = None  # Input image of network
    output = None # Output depth map of combined network
    depth = None  # Ground truth depth map

    for data in dataloaders:
      # Save images to be printed
      input = data['image'].to(device)
      depth = data['depth'].to(device)
      
      output = coarse_net(input)
      loss = criterion(output, depth, 0.5)
      #loss = criterion(output, torch.log(depth))

      coarse_net.zero_grad()
      loss.backward() 

      optimizer.step()
      running_loss += loss.item() #* dataloaders.batch_size

    # Calculate epoch loss
    epoch_loss = running_loss / len(dataloaders)
    epoch_loss_history.append(epoch_loss)

    
    scheduler.step(epoch_loss)
    # Display images and loss
    if not epoch % 10:
      out_max = torch.max(output[0]).item()
      print("Coarse Epoch Default (Log):",epoch)
      display_coarse_epoch(input[0].detach().cpu().numpy()[0], 
                    output[0].detach().cpu().numpy(), 
                    depth[0].detach().cpu().numpy())
      print("Loss:",epoch_loss,'\n')

      print("Coarse Epoch Exp:",epoch)
      display_coarse_epoch(input[0].detach().cpu().numpy()[0], 
                    np.exp(output[0].detach().cpu().numpy()), 
                    depth[0].detach().cpu().numpy())
      print("Loss:",epoch_loss,'\n')


  return epoch_loss_history 


## Fine Train

In [ ]:
def train_fine(fine_net, coarse_net, dataloaders, criterion, optimizer, 
               num_epochs):
  
  scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min',factor=0.1,patience = 10)
  epoch_loss_history = []
  # Batches of images
  input = None  # Input image of network
  output = None # Output depth map of combined network
  coarse_out = None
  depth = None  # Ground truth depth map
  epoch_loss = None

  for epoch in range(num_epochs):
    running_loss = 0.0

    for data in dataloaders:

      coarse_net.eval()

      # Save images to be printed
      input = data['image'].to(device)
      depth = data['depth'].to(device)

      # Pass images through networks
      with torch.no_grad():
            coarse_out = coarse_net(input).unsqueeze(1)

      output = fine_net(input, coarse_out).squeeze(1)
      
      # Update model
      loss = criterion(output, depth, 0.5)
      #loss = criterion(output, torch.log(depth))
      
      fine_net.zero_grad()

      loss.backward() 
      optimizer.step()

      running_loss += loss.item() #* dataloaders.batch_size

    # Calculate epoch loss
    epoch_loss = running_loss / len(dataloaders)
    epoch_loss_history.append(epoch_loss)
    
    coarse_out = coarse_out.squeeze(1)
    scheduler.step(epoch_loss)
    # Display images and loss
    if not epoch % 10:
      print("Full Net Epoch Default (Log):",epoch)
      display_fine_epoch(input[0].detach().cpu().numpy()[0], 
                    coarse_out[0].detach().cpu().numpy(),
                    output[0].detach().cpu().numpy(),
                    depth[0].detach().cpu().numpy())
      print("Loss:",epoch_loss,'\n')

      print("Full Net Epoch Exp:",epoch)
      display_fine_epoch(input[0].detach().cpu().numpy()[0], 
                    np.exp(coarse_out[0].detach().cpu().numpy()),
                    np.exp(output[0].detach().cpu().numpy()),
                    depth[0].detach().cpu().numpy())
      print("Loss:",epoch_loss,'\n')


  # Display final loss and sample images
  print("Final Full Epoch Default (Log):",epoch)
  display_fine_epoch(input[0].detach().cpu().numpy()[0], 
                    coarse_out[0].detach().cpu().numpy(),
                    output[0].detach().cpu().numpy(),
                    depth[0].detach().cpu().numpy())
  display_fine_epoch(input[1].detach().cpu().numpy()[0], 
                    coarse_out[1].detach().cpu().numpy(),
                    output[1].detach().cpu().numpy(),
                    depth[1].detach().cpu().numpy())
  display_fine_epoch(input[2].detach().cpu().numpy()[0], 
                    coarse_out[2].detach().cpu().numpy(),
                    output[2].detach().cpu().numpy(),
                    depth[2].detach().cpu().numpy())
  display_fine_epoch(input[3].detach().cpu().numpy()[0], 
                    coarse_out[3].detach().cpu().numpy(),
                    output[3].detach().cpu().numpy(),
                    depth[3].detach().cpu().numpy())
  print("Final Loss:",epoch_loss,'\n')

  print("Final Full Epoch Exp:",epoch)
  display_fine_epoch(input[0].detach().cpu().numpy()[0], 
                    np.exp(coarse_out[0].detach().cpu().numpy()),
                    np.exp(output[0].detach().cpu().numpy()),
                    depth[0].detach().cpu().numpy())
  display_fine_epoch(input[1].detach().cpu().numpy()[0], 
                    np.exp(coarse_out[1].detach().cpu().numpy()),
                    np.exp(output[1].detach().cpu().numpy()),
                    depth[1].detach().cpu().numpy())
  display_fine_epoch(input[2].detach().cpu().numpy()[0], 
                    np.exp(coarse_out[2].detach().cpu().numpy()),
                    np.exp(output[2].detach().cpu().numpy()),
                    depth[2].detach().cpu().numpy())
  display_fine_epoch(input[3].detach().cpu().numpy()[0], 
                    np.exp(coarse_out[3].detach().cpu().numpy()),
                    np.exp(output[3].detach().cpu().numpy()),
                    depth[3].detach().cpu().numpy())
  print("Final Loss:",epoch_loss,'\n')
  
  return epoch_loss_history

# Optimizer/Loss

In [ ]:
def make_coarse_optimizer(model):
    optimizer = optim.SGD([
                {'params': model.coarse1.parameters()},
                {'params': model.coarse2.parameters()},
                {'params': model.coarse3.parameters()},
                {'params': model.coarse4.parameters()},
                {'params': model.coarse5.parameters()},
                {'params': model.coarse6.parameters(), 'lr': 0.1},
                {'params': model.output.parameters(), 'lr': 0.1}
            ], lr=0.001, momentum=0.9,weight_decay=0.1)
    return optimizer

def make_fine_optimizer(model):
    optimizer = optim.SGD([
                {'params': model.fine1.parameters()},
                {'params': model.fine3.parameters(), 'lr': 0.01},
                {'params': model.fine4.parameters()}
            ], lr=0.001, momentum=0.9,weight_decay=0.1)
    return optimizer

# Scale-invariant Mean Squared Error (in log space)
"""
def SIMSE(y_pred, y_true):
  inner = torch.mean(torch.log(y_true ) - torch.log(y_pred))
  outer = torch.mean( (torch.log(y_pred) - torch.log(y_true) + inner)**2)
  return outer
"""


def SIMSE(y_pred, y_true, L):
  mask = (y_true == 0) | (y_true == y_true.max()) | (y_true == y_true.min())
  d = y_pred[~mask] - torch.log(y_true[~mask])
  n = torch.numel(d)

  d2 = torch.pow(d,2)
  left = torch.sum(d2)/n
  
  right = L * torch.pow(torch.sum(d), 2)/ (n**2)
  loss = left - right
  return loss

def Lin_SIMSE(y_pred, y_true):
  print("Min GT:",torch.min(y_true))
  print(torch.min(y_pred))

  print("Max GT:",torch.max(y_true))
  print(torch.max(y_pred))
  L = 0.5
  n = torch.numel(y_pred)
  d = y_pred - y_true
  d2 = torch.pow(d,2)
  left = torch.sum(d2)/n
  right = L * torch.pow(torch.sum(d), 2)/ (n**2)
  loss = left - right
  return loss

# Data Visualization

In [ ]:
def plot_losses(x_range, vals, title):
  x = np.arange(x_range)
  plt.figure()
  plt.plot(x, vals)
  plt.legend(['Training Losses'])
  plt.xticks(x)
  plt.title(title)
  plt.xlabel('Epoch')
  plt.ylabel('Epoch Loss')
  plt.show()

# Dataloader

In [ ]:
# Select images and depths from the data dictionary
class DepthDataset(Dataset):
  def __init__(self, data_idxs, transform=None, mode='train'):
      self.transform = transform
      self.data_idxs = data_idxs
      self.mode = mode
  def __len__(self):
    return data_dict['depths'][:,:,self.data_idxs].shape[2]

  def __getitem__(self,idx):
    image = data_dict['images'][:,:,:,self.data_idxs][:,:,:,idx]
    depth = data_dict['depths'][:,:,self.data_idxs][:,:,idx]
    label = data_dict['labels'][:,:,self.data_idxs][:,:,idx]

    #sample = {'image': image, 'depth': depth, 'label' : label}
    
    
    if self.mode == 'train':
        
        deg = random.uniform(-5.0,5.0)
        flip = random.uniform(0,1)
        
        image = torchvision.transforms.functional.rotate(transforms.ToPILImage()(image),deg)
        depth = torchvision.transforms.functional.rotate(transforms.ToPILImage()(depth),deg)

        image = np.array(image)
        depth = np.array(depth)

        if flip < 0.5:
          image = torchvision.transforms.functional.hflip(transforms.ToPILImage()(image))
          depth = torchvision.transforms.functional.hflip(transforms.ToPILImage()(depth))


        image = np.array(image)
        depth = np.array(depth)
    
    sample = {'image': image, 'depth': depth, 'label' : label}
    sample = self.transform(sample)

    return sample

"""
Rescale and ToTensor are copied from the pytorch tutorial documentation:
https://pytorch.org/tutorials/beginner/data_loading_tutorial.html
"""

class Rescale(object):
    """Rescale the image in a sample to a given size.
    Args:
        output_size (tuple or int): Desired output size. If tuple, output is
            matched to output_size. If int, smaller of image edges is matched
            to output_size keeping aspect ratio the same.
    """

    def __init__(self, output_size,depth_size):
        assert isinstance(output_size, (int, tuple))
        assert isinstance(depth_size, (int, tuple))
        self.output_size = output_size
        self.depth_size = depth_size
        self.label_size = depth_size

    def __call__(self, sample):
        image, depth, label = sample['image'], sample['depth'], sample['label']

        h, w = image.shape[:2]
        if isinstance(self.output_size, int):
            if h > w:
                new_h, new_w = self.output_size * h / w, self.output_size
            else:
                new_h, new_w = self.output_size, self.output_size * w / h
        else:
            new_h, new_w = self.output_size
       
        new_h, new_w = int(new_h), int(new_w)

        img = transform.resize(image, (new_h, new_w))
        
        h, w = depth.shape
        if isinstance(self.depth_size, int):
            if h > w:
                new_h, new_w = self.depth_size * h / w, self.depth_size
            else:
                new_h, new_w = self.depth_size, self.depth_size * w / h
        else:
            new_h, new_w = self.depth_size

        dep = transform.resize(depth, (new_h, new_w))

        h, w = label.shape
        if isinstance(self.label_size, int):
            if h > w:
                new_h, new_w = self.label_size * h / w, self.label_size
            else:
                new_h, new_w = self.label_size, self.label_size * w / h
        else:
            new_h, new_w = self.label_size

        lab = transform.resize(label, (new_h, new_w))

        return {'image': img, 'depth': dep, 'label' : lab}


class ToTensor(object):
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        image, depth, label = sample['image'], sample['depth'], sample['label']

        # swap color axis because
        # numpy image: H x W x C
        # torch image: C X H X W
        image = image.transpose((2, 0, 1))
        return {'image': torch.from_numpy(np.float32(image)),
                'depth': torch.from_numpy(np.float32(depth)),
                'label': torch.from_numpy(np.float32(label))}

# Train Full

In [ ]:
# Train network
model_path = 'drive/MyDrive/E442-Final-Project-Data/'
ans = input("Do you want to train a new network?")
if ans == 'y':
    print("Training a new network...")
    num_epochs = 41
    criterion = SIMSE
    #criterion = nn.MSELoss()

    # Load Data
    dataset = DepthDataset(train_large, 
                        transform=transforms.Compose([
                                Rescale((228, 304),(55, 74)),
                                ToTensor()]), mode='train')
    
    data_loader = DataLoader(dataset,batch_size=32,shuffle=True,num_workers=0)

    # Train Coarse Network
    coarseNet = Coarse_Network()
    coarseNet.cuda()
    coarse_optimizer = make_coarse_optimizer(coarseNet)
    epoch_loss_history_coarse = train_coarse(coarseNet, data_loader, criterion, coarse_optimizer, num_epochs)

    # Train Fine Network
    fineNet = Fine_Network()
    fineNet.cuda()
    fine_optimizer = make_fine_optimizer(fineNet)
    epoch_loss_history_fine = train_fine(fineNet, coarseNet, data_loader, criterion, fine_optimizer, num_epochs)

    plot_losses(num_epochs, epoch_loss_history_coarse, "NYUDepth_Coarse")
    plot_losses(num_epochs, epoch_loss_history_fine, "NYUDepth_Combined")

    # Save network
    ans = input("Do you want to save this model? (y/n)")
    if ans == 'y':
        # Save current network
        ans = ''
        while ans == '':
            ans = input("Enter model name")
        dataSet = input("Which dataset are you using? (Fx: NYUDepthV1)")
        torch.save({
                    'epoch': num_epochs,
                    'fine_model_state_dict': fineNet.state_dict(),
                    'coarse_model_state_dict': coarseNet.state_dict(),
                    'optimizer_state_dict': fine_optimizer.state_dict(),
                    'loss': epoch_loss_history_fine,
                    'dataSet': dataSet,
                    'train_idxs': train_small,
                    'val_idxs': val_small,
                    }, model_path + ans + '.model')
    model = fineNet



# Load and Validate

In [ ]:
checkpoint = None
coarseNet = Coarse_Network().cuda()
fineNet = Fine_Network().cuda()

# Load in model
ans = input("Do you want to load an existing model? (y/n)")
model_path = 'drive/MyDrive/E442-Final-Project-Data/'
if ans == 'y':

    criterion = SIMSE

    # Find models
    files = os.listdir(model_path)
    models = [f[:-6] for f in files if f[-6:] == '.model']

    # Pick model
    if len(models) != 0:
        print(models)
        while ans not in models:
            ans = input("Enter model: ")
        checkpoint = torch.load(model_path + ans + '.model')
    else:
        print("There are no models...")
        sys.exit()

    # Validation
    ans = input("Do you want to test on the validation and training set? (y/n)")
    if ans == 'y':
        # Transform for NYUDepth
        trans = transforms.Compose([
                                Rescale((228, 304),(55, 74)),
                                ToTensor()])
        
        # Load model and set to eval for validation
        coarseNet.load_state_dict(checkpoint['coarse_model_state_dict'])
        fineNet.load_state_dict(checkpoint['fine_model_state_dict'])
        coarseNet.eval()
        fineNet.eval()

        # Calculate training loss
        # train_idxs = checkpoint['train_idxs']
        train_idxs = train_large
        
        train_coarse_loss = []
        train_fine_loss = []

        # Create a data set from certain images and depths
        #pretty surue dataset is effectively a subset of data_dict; only contains elements at the indices within train_idxs
        #we honestly didn't need to bother with a dataloader like this? coulda just used loops and broadcasting and been fine? woulda been more straightforward?
        dataset = DepthDataset(train_idxs, 
                        transform=transforms.Compose([
                                Rescale((228, 304),(55, 74)),
                                ToTensor()]),mode='test')
        

        # Create a DataLoader to properly access the data set
        data_loader = DataLoader(dataset,batch_size=1,shuffle=False,num_workers=0)

        for datums in data_loader:
            # Extract image and depth
            image = datums['image'].to(device)
            depth = datums['depth'].to(device)

            # Pass image/depth into networks
            coarse_out = coarseNet(image)
            fine_out = fineNet(image, coarse_out.unsqueeze(1)).squeeze(1)

            # Calculate loss, L=1 indicates SIError(see Eigen et al. top of pg5)
            fine_loss = criterion(fine_out, depth, L=1)
            train_fine_loss.append(fine_loss.item())
            coarse_loss = criterion(coarse_out, depth, L=1)
            train_coarse_loss.append(coarse_loss.item())

        print("Average training loss of coarse network:", np.array(train_coarse_loss).mean())
        print("Average training loss of coarse + fine network:", np.array(train_fine_loss).mean())

        # Calculate validation loss
        # val_idxs = checkpoint['val_idxs']
        val_idxs = val_large
        
        val_coarse_loss = []
        val_fine_loss = []
        val_coarse_RMSE_loss = []
        val_fine_RMSE_loss = []

        val_input_img = []
        val_coarse_img = []
        val_fine_img = []
        val_GT_img = []


        # Create a data set from certain images and depths
        #pretty surue dataset is effectively a subset of data_dict; only contains elements at the indices within train_idxs
        #we honestly didn't need to bother with a dataloader like this? coulda just used loops and broadcasting and been fine? woulda been more straightforward?
        dataset = DepthDataset(val_idxs, 
                        transform=transforms.Compose([
                                Rescale((228, 304),(55, 74)),
                                ToTensor()]),mode='test')
        

        # Create a DataLoader to properly access the data set
        data_loader = DataLoader(dataset,batch_size=1,shuffle=False,num_workers=0)

        for datums in data_loader:
            # Extract image and depth
            image = datums['image'].to(device)
            depth = datums['depth'].to(device)

            # Pass image/depth into networks
            coarse_out = coarseNet(image)
            fine_out = fineNet(image, coarse_out.unsqueeze(1)).squeeze(1)

            # Calculate loss, L=1 indicates SIError(see Eigen et al. top of pg5)
            fine_loss = criterion(fine_out, depth, L=1)
            val_fine_loss.append(fine_loss.item())
            coarse_loss = criterion(coarse_out, depth, L=1)
            val_coarse_loss.append(coarse_loss.item())

            # Calculate loss for RMSE
            fine_loss = torch.sqrt( nn.functional.mse_loss(fine_out, torch.log(depth)) )
            val_fine_RMSE_loss.append(fine_loss.item())
            coarse_loss = torch.sqrt( nn.functional.mse_loss(coarse_out, torch.log(depth)) )
            val_coarse_RMSE_loss.append(coarse_loss.item())

            # Save Output Images
            val_input_img.append(image)
            val_coarse_img.append(coarse_out)
            val_fine_img.append(fine_out)
            val_GT_img.append(depth)

            # # Calculate loss
            # loss = criterion(fine_out, depth, L=0.5)
            # val_loss.append(loss.item())
        
        # Print losses
        print("Average validation loss of coarse network:", np.array(val_coarse_loss).mean())
        print("Average validation loss of coarse + fine network:", np.array(val_fine_loss).mean())
        print("Average validation RMSE loss of coarse network:", np.array(val_coarse_RMSE_loss).mean())
        print("Average validation RMSE loss of coarse + fine network:", np.array(val_fine_RMSE_loss).mean())


      

In [ ]:
def display_results(input, coarse_output, fine_output, GT):
  rows = 1
  columns = 4 
  fig = plt.figure(figsize=(12, 5))

  # Adds a subplot at the 1st position
  fig.add_subplot(rows, columns, 1)
  
  # Show image
  plt.imshow(cv2.cvtColor(input, cv2.COLOR_BGR2RGB))
  plt.axis('off')
  
  # Adds a subplot at the 2nd position
  fig.add_subplot(rows, columns, 2)
  
  # Show image
  plt.imshow(coarse_output,cmap='jet_r')
  plt.axis('off')

  # Adds a subplot at the 3rd position
  fig.add_subplot(rows, columns, 3)

  # Show image
  plt.imshow(fine_output,cmap='jet_r')
  plt.axis('off')

  # Adds a subplot at the 4th position
  fig.add_subplot(rows, columns, 4)
  
  # Show image
  plt.imshow(GT,cmap='jet_r')
  plt.axis('off')

  plt.show()



"""

print("Validation Output")
v_loss = np.array(val_loss)
idx = np.argpartition(v_loss, 129)
display_fine_epoch(val_input_img[48][0].detach().cpu().numpy()[0], 
                    np.exp(val_coarse_img[48][0][0].detach().cpu().numpy()),
                    np.exp(val_fine_img[48][0].detach().cpu().numpy()),
                    val_GT_img[48][0].detach().cpu().numpy())
good_idx = [45,130,131,60,67,120,86,138,96]
for i in good_idx:
  #i = idx[k]
  #print(i,":")
  display_results(val_input_img[i][0].detach().cpu().numpy()[0], 
                    np.exp(val_coarse_img[i][0][0].detach().cpu().numpy()),
                    np.exp(val_fine_img[i][0].detach().cpu().numpy()),
                    val_GT_img[i][0].detach().cpu().numpy())
"""
print(np.mean(val_loss), np.std(val_loss), np.percentile(val_loss, 90))
print(np.mean(train_loss), np.std(train_loss), np.percentile(train_loss, 90))
